# Looking for the best Formula 1 season

For my master's project, I'm making a piece about answering the question: **What championship winning team had the best Formula 1 season?**

To do this I was working with data provided by the [Ergast Developer API](https://ergast.com/mrd/). I noticed an error in the driver-constructor pairing for the 1950 season and wanted to verify things without moving forward. I was originally going to create a table of the driver-constructor pairs for each race, and then compare it with the data I had.

Instead I went straight to the source for F1 information, [formula1.com](https://formula1.com), and scraped race information for each race from 1950 to 2018.

In [1]:
import pandas as pd
import numpy as np

The first thing to do is to import the data

In [2]:
race_results = pd.read_csv("../data/from_scripts/race_results.csv")

In [3]:
race_results.head()

,race_id,year,round_id,race_date,race_name,driver,code,constructor,position,position_order,laps,time,points
0,1,1950,1,13 May 1950,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,9.0
1,1,1950,1,13 May 1950,Great Britain,Luigi Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,6.0
2,1,1950,1,13 May 1950,Great Britain,Reg Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,4.0
3,1,1950,1,13 May 1950,Great Britain,Yves Giraud Cabantous,CAB,Talbot-Lago,4,4,68.0,+2 laps,3.0
4,1,1950,1,13 May 1950,Great Britain,Louis Rosier,ROS,Talbot-Lago,5,5,68.0,+2 laps,2.0


In [4]:
race_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22395 entries, 0 to 22394
Data columns (total 13 columns):
race_id           22395 non-null int64
year              22395 non-null int64
round_id          22395 non-null int64
race_date         22395 non-null object
race_name         22395 non-null object
driver            22395 non-null object
code              22395 non-null object
constructor       22369 non-null object
position          22395 non-null object
position_order    22395 non-null int64
laps              22171 non-null float64
time              22387 non-null object
points            22395 non-null float64
dtypes: float64(2), int64(4), object(7)
memory usage: 2.2+ MB


Let's also check how many races we have:

In [5]:
race_results.race_id.max()

997

Things seem to be in good order.

Using the `race_results.csv` I also made a clean list of constructor names that I want to add to the `race_results` dataFrame.

In [6]:
constructors_all = pd.read_csv("../data/other/constructors_clean.csv")

In [7]:
constructors_all.head()

,year,constructor,constructor_clean,constructor_alt
0,1950,Adams Offenhauser,Adams,NaN
1,1950,Alfa Romeo,Alfa Romeo,NaN
2,1950,Alta,Alta,NaN
3,1950,Cooper JAP,Cooper,NaN
4,1950,Deidt Offenhauser,Deidt,NaN


In [8]:
race_results = race_results.merge(constructors_all, on=["year", "constructor"])

In [9]:
race_results.sort_values(["race_id", "position_order"]).head(10)

,race_id,year,round_id,race_date,race_name,driver,code,constructor,position,position_order,laps,time,points,constructor_clean,constructor_alt
0,1,1950,1,13 May 1950,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,9.0,Alfa Romeo,NaN
1,1,1950,1,13 May 1950,Great Britain,Luigi Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,6.0,Alfa Romeo,NaN
2,1,1950,1,13 May 1950,Great Britain,Reg Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,4.0,Alfa Romeo,NaN
22,1,1950,1,13 May 1950,Great Britain,Yves Giraud Cabantous,CAB,Talbot-Lago,4,4,68.0,+2 laps,3.0,Talbot-Lago,NaN
23,1,1950,1,13 May 1950,Great Britain,Louis Rosier,ROS,Talbot-Lago,5,5,68.0,+2 laps,2.0,Talbot-Lago,NaN
59,1,1950,1,13 May 1950,Great Britain,Bob Gerard,GER,ERA,6,6,67.0,+3 laps,0.0,ERA,NaN
60,1,1950,1,13 May 1950,Great Britain,Cuth Harrison,HAR,ERA,7,7,67.0,+3 laps,0.0,ERA,NaN
24,1,1950,1,13 May 1950,Great Britain,Philippe Etancelin,ETA,Talbot-Lago,8,8,65.0,+5 laps,0.0,Talbot-Lago,NaN
67,1,1950,1,13 May 1950,Great Britain,David Hampshire,HAM,Maserati,9,9,64.0,+6 laps,0.0,Maserati,NaN
68,1,1950,1,13 May 1950,Great Britain,Joe Fry,FRY,Maserati,10,10,NaN,SHC,0.0,Maserati,NaN


For the majority of the analysis, I'll only be looking at the championship runs, a subset of all these results. Let's cut a slice of the results that is only of the winning runs each year:

In [10]:
winning_teams = pd.read_csv("../data/from_scripts/winning_teams.csv")

In [11]:
winning_teams.head()

,year,constructor
0,1950,Alfa Romeo
1,1951,Alfa Romeo
2,1952,Ferrari
3,1953,Ferrari
4,1954,Mercedes-Benz


Let's connect the `race_results` and `winning_teams`

In [12]:
combine = pd.merge(race_results, winning_teams, how="left", on=["year", "constructor"], indicator="winners")

In [13]:
combine.head()

,race_id,year,round_id,race_date,race_name,driver,code,constructor,position,position_order,laps,time,points,constructor_clean,constructor_alt,winners
0,1,1950,1,13 May 1950,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,9.0,Alfa Romeo,NaN,both
1,1,1950,1,13 May 1950,Great Britain,Luigi Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,6.0,Alfa Romeo,NaN,both
2,1,1950,1,13 May 1950,Great Britain,Reg Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,4.0,Alfa Romeo,NaN,both
3,1,1950,1,13 May 1950,Great Britain,Juan Manuel Fangio,FAN,Alfa Romeo,NC,13,62.0,DNF,0.0,Alfa Romeo,NaN,both
4,2,1950,2,21 May 1950,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,9.0,Alfa Romeo,NaN,both


In [14]:
results = combine[combine.winners == "both"]
results.head(20)

,race_id,year,round_id,race_date,race_name,driver,code,constructor,position,position_order,laps,time,points,constructor_clean,constructor_alt,winners
0,1,1950,1,13 May 1950,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,9.0,Alfa Romeo,NaN,both
1,1,1950,1,13 May 1950,Great Britain,Luigi Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,6.0,Alfa Romeo,NaN,both
2,1,1950,1,13 May 1950,Great Britain,Reg Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,4.0,Alfa Romeo,NaN,both
3,1,1950,1,13 May 1950,Great Britain,Juan Manuel Fangio,FAN,Alfa Romeo,NC,13,62.0,DNF,0.0,Alfa Romeo,NaN,both
4,2,1950,2,21 May 1950,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,9.0,Alfa Romeo,NaN,both
5,2,1950,2,21 May 1950,Monaco,Luigi Fagioli,FAG,Alfa Romeo,NC,11,0.0,DNF,0.0,Alfa Romeo,NaN,both
6,2,1950,2,21 May 1950,Monaco,Nino Farina,FAR,Alfa Romeo,NC,12,0.0,DNF,0.0,Alfa Romeo,NaN,both
7,4,1950,4,04 Jun 1950,Switzerland,Nino Farina,FAR,Alfa Romeo,1,1,42.0,2:02:53.700,9.0,Alfa Romeo,NaN,both
8,4,1950,4,04 Jun 1950,Switzerland,Luigi Fagioli,FAG,Alfa Romeo,2,2,42.0,+0.400s,6.0,Alfa Romeo,NaN,both
9,4,1950,4,04 Jun 1950,Switzerland,Juan Manuel Fangio,FAN,Alfa Romeo,NC,12,33.0,DNF,0.0,Alfa Romeo,NaN,both


Looking at this, I can go ahead and drop some columns that aren't immediately useful:

* `race_date`
* `points`
* `winners`
* `constructor_alt`

In [15]:
results = results.drop(columns=["race_date", "points", "winners", "constructor_alt"]) 

In [16]:
results.head()

,race_id,year,round_id,race_name,driver,code,constructor,position,position_order,laps,time,constructor_clean
0,1,1950,1,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,Alfa Romeo
1,1,1950,1,Great Britain,Luigi Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,Alfa Romeo
2,1,1950,1,Great Britain,Reg Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,Alfa Romeo
3,1,1950,1,Great Britain,Juan Manuel Fangio,FAN,Alfa Romeo,NC,13,62.0,DNF,Alfa Romeo
4,2,1950,2,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,Alfa Romeo


In [17]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2293 entries, 0 to 22058
Data columns (total 12 columns):
race_id              2293 non-null int64
year                 2293 non-null int64
round_id             2293 non-null int64
race_name            2293 non-null object
driver               2293 non-null object
code                 2293 non-null object
constructor          2293 non-null object
position             2293 non-null object
position_order       2293 non-null int64
laps                 2267 non-null float64
time                 2293 non-null object
constructor_clean    2293 non-null object
dtypes: float64(1), int64(4), object(7)
memory usage: 232.9+ KB


Now we can head into the analysis.

---

## Method 01: Wins

The first way we can look to find an answer to the question is to look at the wins each team had in their run.

In [18]:
wins = results[results.position == "1"]

In [19]:
wins.head()

,race_id,year,round_id,race_name,driver,code,constructor,position,position_order,laps,time,constructor_clean
0,1,1950,1,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,Alfa Romeo
4,2,1950,2,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,Alfa Romeo
7,4,1950,4,Switzerland,Nino Farina,FAR,Alfa Romeo,1,1,42.0,2:02:53.700,Alfa Romeo
10,5,1950,5,Belgium,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,35.0,2:47:26.000,Alfa Romeo
13,6,1950,6,France,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,64.0,2:57:52.800,Alfa Romeo


In [20]:
wins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 528 entries, 0 to 22057
Data columns (total 12 columns):
race_id              528 non-null int64
year                 528 non-null int64
round_id             528 non-null int64
race_name            528 non-null object
driver               528 non-null object
code                 528 non-null object
constructor          528 non-null object
position             528 non-null object
position_order       528 non-null int64
laps                 526 non-null float64
time                 528 non-null object
constructor_clean    528 non-null object
dtypes: float64(1), int64(4), object(7)
memory usage: 53.6+ KB


In [21]:
wins.head(15)

,race_id,year,round_id,race_name,driver,code,constructor,position,position_order,laps,time,constructor_clean
0,1,1950,1,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,Alfa Romeo
4,2,1950,2,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,Alfa Romeo
7,4,1950,4,Switzerland,Nino Farina,FAR,Alfa Romeo,1,1,42.0,2:02:53.700,Alfa Romeo
10,5,1950,5,Belgium,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,35.0,2:47:26.000,Alfa Romeo
13,6,1950,6,France,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,64.0,2:57:52.800,Alfa Romeo
16,7,1950,7,Italy,Nino Farina,FAR,Alfa Romeo,1,1,80.0,2:51:17.400,Alfa Romeo
158,8,1951,1,Switzerland,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,42.0,2:07:53.640,Alfa Romeo
162,10,1951,3,Belgium,Nino Farina,FAR,Alfa Romeo,1,1,36.0,2:45:46.200,Alfa Romeo
165,11,1951,4,France,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,77.0,3:22:11.000,Alfa Romeo
166,11,1951,4,France,Luigi Fagioli,FAG,Alfa Romeo,1,2,NaN,SHC,Alfa Romeo


One thing with the earlier F1 races that I've noted in my previous analysis attempts is that the there were a lot of shared drives. In this dataset, those rows are easier to find. In the column for time, they have a value of `SHC`. In doing my calculations, I'll ignore these rows.

In [22]:
wins_clean = wins[wins["time"] != "SHC"]

In [23]:
wins_clean.head(15)

,race_id,year,round_id,race_name,driver,code,constructor,position,position_order,laps,time,constructor_clean
0,1,1950,1,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,Alfa Romeo
4,2,1950,2,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,Alfa Romeo
7,4,1950,4,Switzerland,Nino Farina,FAR,Alfa Romeo,1,1,42.0,2:02:53.700,Alfa Romeo
10,5,1950,5,Belgium,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,35.0,2:47:26.000,Alfa Romeo
13,6,1950,6,France,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,64.0,2:57:52.800,Alfa Romeo
16,7,1950,7,Italy,Nino Farina,FAR,Alfa Romeo,1,1,80.0,2:51:17.400,Alfa Romeo
158,8,1951,1,Switzerland,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,42.0,2:07:53.640,Alfa Romeo
162,10,1951,3,Belgium,Nino Farina,FAR,Alfa Romeo,1,1,36.0,2:45:46.200,Alfa Romeo
165,11,1951,4,France,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,77.0,3:22:11.000,Alfa Romeo
184,15,1951,9,Spain,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,70.0,2:46:54.100,Alfa Romeo


In [24]:
wins_grouped = wins_clean.copy().groupby(["year", "constructor_clean"])

In [25]:
count_wins = wins_grouped.position.count().rename("wins")

In [26]:
count_wins.head(20)

year  constructor_clean
1950  Alfa Romeo           6
1951  Alfa Romeo           4
1952  Ferrari              7
1953  Ferrari              7
1954  Mercedes             4
1955  Mercedes             5
1956  Ferrari              5
1957  Maserati             4
1958  Ferrari              2
1959  Cooper               5
1960  Cooper               6
1961  Ferrari              5
1962  BRM                  4
1963  Lotus                7
1964  Ferrari              3
1965  Lotus                6
1966  Brabham              4
1967  Brabham              4
1968  Lotus                5
1969  Matra                6
Name: wins, dtype: int64

In [27]:
count_wins = count_wins.to_frame().reset_index()

In [28]:
count_wins.head(13)

,year,constructor_clean,wins
0,1950,Alfa Romeo,6
1,1951,Alfa Romeo,4
2,1952,Ferrari,7
3,1953,Ferrari,7
4,1954,Mercedes,4
5,1955,Mercedes,5
6,1956,Ferrari,5
7,1957,Maserati,4
8,1958,Ferrari,2
9,1959,Cooper,5


In [29]:
count_wins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 3 columns):
year                 69 non-null int64
constructor_clean    69 non-null object
wins                 69 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.7+ KB


In [30]:
count_wins.sort_values(by="wins",ascending=False)

,year,constructor_clean,wins
66,2016,Mercedes,19
65,2015,Mercedes,16
64,2014,Mercedes,16
38,1988,McLaren,15
54,2004,Ferrari,15
52,2002,Ferrari,15
63,2013,Red Bull,13
34,1984,McLaren,12
67,2017,Mercedes,12
61,2011,Red Bull,12


Let me add an extra column for the run to reference and then we can save it:

In [31]:
def team_run(row):
    return (str(row.year) + " " + str(row.constructor_clean))
    
count_wins["run"] = count_wins.apply(team_run, axis=1)

In [32]:
count_wins.head(3)

,year,constructor_clean,wins,run
0,1950,Alfa Romeo,6,1950 Alfa Romeo
1,1951,Alfa Romeo,4,1951 Alfa Romeo
2,1952,Ferrari,7,1952 Ferrari


In [33]:
count_wins.to_csv("../data/output/win_count.csv",index=False)

Looking at this list, Mercedes' 2016 run would be the best F1 season. But we're not accounting for the fact that the number of races per season has varied. Let's try to take that into account by finding the number of races in a season and dividing the number of wins by the number of races in the season.

In [34]:
normalize_wins = count_wins.copy()

In [35]:
def races_in_season(row):
    y = int(row.year)
    season = results[results.year == y]
    number = season.round_id.max()
    return number

In [36]:
normalize_wins["races"] = normalize_wins.apply(races_in_season, axis=1)

In [37]:
normalize_wins.head()

,year,constructor_clean,wins,run,races
0,1950,Alfa Romeo,6,1950 Alfa Romeo,7
1,1951,Alfa Romeo,4,1951 Alfa Romeo,9
2,1952,Ferrari,7,1952 Ferrari,8
3,1953,Ferrari,7,1953 Ferrari,9
4,1954,Mercedes,4,1954 Mercedes,9


Now we can find the percentage of races each team had in their season:

In [38]:
normalize_wins["win_percentage"] = (normalize_wins["wins"] / normalize_wins["races"])*100

In [39]:
normalize_wins.head()

,year,constructor_clean,wins,run,races,win_percentage
0,1950,Alfa Romeo,6,1950 Alfa Romeo,7,85.714286
1,1951,Alfa Romeo,4,1951 Alfa Romeo,9,44.444444
2,1952,Ferrari,7,1952 Ferrari,8,87.500000
3,1953,Ferrari,7,1953 Ferrari,9,77.777778
4,1954,Mercedes,4,1954 Mercedes,9,44.444444


And now we can sort the runs based on win percentages

In [40]:
normalize_wins.sort_values(by="win_percentage",ascending=False)

,year,constructor_clean,wins,run,races,win_percentage
38,1988,McLaren,15,1988 McLaren,16,93.750000
66,2016,Mercedes,19,2016 Mercedes,21,90.476190
52,2002,Ferrari,15,2002 Ferrari,17,88.235294
2,1952,Ferrari,7,1952 Ferrari,8,87.500000
0,1950,Alfa Romeo,6,1950 Alfa Romeo,7,85.714286
65,2015,Mercedes,16,2015 Mercedes,19,84.210526
64,2014,Mercedes,16,2014 Mercedes,19,84.210526
54,2004,Ferrari,15,2004 Ferrari,18,83.333333
3,1953,Ferrari,7,1953 Ferrari,9,77.777778
46,1996,Williams,12,1996 Williams,16,75.000000


From this we can see that McLaren's 1988 run was actually better than Mercedes' 2016 run — McLaren only lost 1 race. Let's save it.

In [41]:
normalize_wins.to_csv("../data/output/win_count_normalized.csv", index=False)

Let's move on to method 2.

---

## Method 02: Podiums

Looking at the wins is a good start, but there are a lot of factors about the team's performance over a season that it leaves out.

* It only shows a very narrow slice of the team's drivers's performance. If we only know that one of the drivers won, we have no idea how the other driver did.
* It offers a limited amount of comparison. Winning is a binary variable — you win or you don't. When looking at the history of the sport, things are greyer. For example, Keke Rosberg won the driver's cup in 1982, but he only had one victory that season. Looking only at the number of wins doesn't provide any context about how this happened.

We can dig a little deeper and look at podiums. The podium refers to the drivers who finished first, second, and third in any given race. A team that consistenly has both drivers on the podium over a season is doing amazing. (ex: Mercedes in 2019 is dominating right now with Bottas and Hamilton in the podium for every race)

In [42]:
podiums = results[results.position.isin(["1", "2", "3"])]

In [43]:
podiums.head(23)

,race_id,year,round_id,race_name,driver,code,constructor,position,position_order,laps,time,constructor_clean
0,1,1950,1,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,Alfa Romeo
1,1,1950,1,Great Britain,Luigi Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,Alfa Romeo
2,1,1950,1,Great Britain,Reg Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,Alfa Romeo
4,2,1950,2,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,Alfa Romeo
7,4,1950,4,Switzerland,Nino Farina,FAR,Alfa Romeo,1,1,42.0,2:02:53.700,Alfa Romeo
8,4,1950,4,Switzerland,Luigi Fagioli,FAG,Alfa Romeo,2,2,42.0,+0.400s,Alfa Romeo
10,5,1950,5,Belgium,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,35.0,2:47:26.000,Alfa Romeo
11,5,1950,5,Belgium,Luigi Fagioli,FAG,Alfa Romeo,2,2,35.0,+14.000s,Alfa Romeo
13,6,1950,6,France,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,64.0,2:57:52.800,Alfa Romeo
14,6,1950,6,France,Luigi Fagioli,FAG,Alfa Romeo,2,2,64.0,+25.700s,Alfa Romeo


As I mentioned earlier, shared drives were a common thing in the early years of Formula1 and they're findable in the data by looking for any rows for where the `time` is `SHC`. 

When looking at the podiums, I'll be doing the same. 

In [44]:
podiums_clean = podiums[podiums.time != "SHC"]

In [45]:
podiums_clean.head(21)

,race_id,year,round_id,race_name,driver,code,constructor,position,position_order,laps,time,constructor_clean
0,1,1950,1,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,Alfa Romeo
1,1,1950,1,Great Britain,Luigi Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,Alfa Romeo
2,1,1950,1,Great Britain,Reg Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,Alfa Romeo
4,2,1950,2,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,Alfa Romeo
7,4,1950,4,Switzerland,Nino Farina,FAR,Alfa Romeo,1,1,42.0,2:02:53.700,Alfa Romeo
8,4,1950,4,Switzerland,Luigi Fagioli,FAG,Alfa Romeo,2,2,42.0,+0.400s,Alfa Romeo
10,5,1950,5,Belgium,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,35.0,2:47:26.000,Alfa Romeo
11,5,1950,5,Belgium,Luigi Fagioli,FAG,Alfa Romeo,2,2,35.0,+14.000s,Alfa Romeo
13,6,1950,6,France,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,64.0,2:57:52.800,Alfa Romeo
14,6,1950,6,France,Luigi Fagioli,FAG,Alfa Romeo,2,2,64.0,+25.700s,Alfa Romeo


Now let's count them.

In [46]:
count_podiums = podiums_clean.groupby(["year", "constructor_clean"]).position.count().rename("podiums")

In [47]:
count_podiums.head()

year  constructor_clean
1950  Alfa Romeo           12
1951  Alfa Romeo            9
1952  Ferrari              17
1953  Ferrari              16
1954  Mercedes              7
Name: podiums, dtype: int64

In [48]:
count_podiums = count_podiums.to_frame().reset_index()

In [49]:
count_podiums.head()

,year,constructor_clean,podiums
0,1950,Alfa Romeo,12
1,1951,Alfa Romeo,9
2,1952,Ferrari,17
3,1953,Ferrari,16
4,1954,Mercedes,7


In [50]:
count_podiums.sort_values(by="podiums",ascending=False).head(20)

,year,constructor_clean,podiums
66,2016,Mercedes,33
65,2015,Mercedes,32
64,2014,Mercedes,31
54,2004,Ferrari,29
52,2002,Ferrari,27
61,2011,Red Bull,27
67,2017,Mercedes,26
38,1988,McLaren,25
68,2018,Mercedes,25
51,2001,Ferrari,24


We can save this version of raw podium counts and get to normalizing the podiums over the season to better compare the different runs. But before that let's add the column to indicate each team's run:

In [51]:
count_podiums["run"] = count_podiums.apply(team_run, axis=1)

In [52]:
count_podiums.to_csv("../data/output/podium_count.csv")

To normalize the podiums, we'll need to get the number of possible podiums in a season. That's the number of races times 3. When counting the number of wins we figured out how to do the races already.

In [53]:
normalize_podiums = count_podiums.copy()

In [54]:
normalize_podiums["races"] = normalize_podiums.apply(races_in_season, axis=1)

In [55]:
normalize_podiums.head()

,year,constructor_clean,podiums,run,races
0,1950,Alfa Romeo,12,1950 Alfa Romeo,7
1,1951,Alfa Romeo,9,1951 Alfa Romeo,9
2,1952,Ferrari,17,1952 Ferrari,8
3,1953,Ferrari,16,1953 Ferrari,9
4,1954,Mercedes,7,1954 Mercedes,9


While there are three podium spots every race, for the majority of Formula 1 seasons, teams have only had two drivers, so the maximum number of podium spots a team can have in a season is `2 * (number of races)`

In [56]:
normalize_podiums["available_podium_spots"] = normalize_podiums.races * 2

In [57]:
normalize_podiums.head()

,year,constructor_clean,podiums,run,races,available_podium_spots
0,1950,Alfa Romeo,12,1950 Alfa Romeo,7,14
1,1951,Alfa Romeo,9,1951 Alfa Romeo,9,18
2,1952,Ferrari,17,1952 Ferrari,8,16
3,1953,Ferrari,16,1953 Ferrari,9,18
4,1954,Mercedes,7,1954 Mercedes,9,18


And now we have everything to compute a podium percentage.

In [58]:
normalize_podiums["podium_percentage"] = (normalize_podiums.podiums / normalize_podiums.available_podium_spots ) * 100

In [59]:
normalize_podiums.head(20)

,year,constructor_clean,podiums,run,races,available_podium_spots,podium_percentage
0,1950,Alfa Romeo,12,1950 Alfa Romeo,7,14,85.714286
1,1951,Alfa Romeo,9,1951 Alfa Romeo,9,18,50.000000
2,1952,Ferrari,17,1952 Ferrari,8,16,106.250000
3,1953,Ferrari,16,1953 Ferrari,9,18,88.888889
4,1954,Mercedes,7,1954 Mercedes,9,18,38.888889
5,1955,Mercedes,10,1955 Mercedes,7,14,71.428571
6,1956,Ferrari,10,1956 Ferrari,9,18,55.555556
7,1957,Maserati,10,1957 Maserati,9,18,55.555556
8,1958,Ferrari,14,1958 Ferrari,11,22,63.636364
9,1959,Cooper,13,1959 Cooper,9,18,72.222222


In [60]:
normalize_podiums.sort_values(by="podium_percentage",ascending=False).head(10)

,year,constructor_clean,podiums,run,races,available_podium_spots,podium_percentage
2,1952,Ferrari,17,1952 Ferrari,8,16,106.250000
11,1961,Ferrari,14,1961 Ferrari,7,14,100.000000
3,1953,Ferrari,16,1953 Ferrari,9,18,88.888889
0,1950,Alfa Romeo,12,1950 Alfa Romeo,7,14,85.714286
65,2015,Mercedes,32,2015 Mercedes,19,38,84.210526
64,2014,Mercedes,31,2014 Mercedes,19,38,81.578947
54,2004,Ferrari,29,2004 Ferrari,18,36,80.555556
52,2002,Ferrari,27,2002 Ferrari,17,34,79.411765
66,2016,Mercedes,33,2016 Mercedes,21,42,78.571429
38,1988,McLaren,25,1988 McLaren,16,32,78.125000


The top four teams all had more than two drivers in their runs. This was a more common thing in the earlier years of Formula 1.

Maybe I should normalize these percentages by accounting for the number of drivers per team in each season. Something like:  
`(podiums / podium_spots) / (number of drivers)`.

To do this, I first need to get a full count of the teams in each season and how many drivers they have race in a season.

In [61]:
results.head()

,race_id,year,round_id,race_name,driver,code,constructor,position,position_order,laps,time,constructor_clean
0,1,1950,1,Great Britain,Nino Farina,FAR,Alfa Romeo,1,1,70.0,2:13:23.600,Alfa Romeo
1,1,1950,1,Great Britain,Luigi Fagioli,FAG,Alfa Romeo,2,2,70.0,+2.600s,Alfa Romeo
2,1,1950,1,Great Britain,Reg Parnell,PAR,Alfa Romeo,3,3,70.0,+52.000s,Alfa Romeo
3,1,1950,1,Great Britain,Juan Manuel Fangio,FAN,Alfa Romeo,NC,13,62.0,DNF,Alfa Romeo
4,2,1950,2,Monaco,Juan Manuel Fangio,FAN,Alfa Romeo,1,1,100.0,3:13:18.700,Alfa Romeo


I had come up with a few different ways of doing this: 

* v1: count the total number of unique drivers in a season regardless of how many races they competed in
* v2: count the number of unique drivers that raced in the team's first race of the season

In [62]:
def drivers_in_season_v1(row):
    season = results[(results.year == row.year)]
    team = season[(season.constructor_clean == row.constructor_clean)]
    
    return team.driver.nunique()

def drivers_in_season_v2(row):
    season = results[results.year == row.year]
    team = season[season.constructor_clean == row.constructor_clean]
    first_race = team.round_id.min()
    
    return team[team.round_id == first_race].driver.nunique()

In [63]:
normalize_podiums["num_drivers_v1"] = normalize_podiums.apply(drivers_in_season_v1, axis=1)
normalize_podiums["num_drivers_v2"] = normalize_podiums.apply(drivers_in_season_v2, axis=1)

In [64]:
normalize_podiums.head(20)

,year,constructor_clean,podiums,run,races,available_podium_spots,podium_percentage,num_drivers_v1,num_drivers_v2
0,1950,Alfa Romeo,12,1950 Alfa Romeo,7,14,85.714286,6,4
1,1951,Alfa Romeo,9,1951 Alfa Romeo,9,18,50.000000,7,4
2,1952,Ferrari,17,1952 Ferrari,8,16,106.250000,15,6
3,1953,Ferrari,16,1953 Ferrari,9,18,88.888889,11,4
4,1954,Mercedes,7,1954 Mercedes,9,18,38.888889,4,3
5,1955,Mercedes,10,1955 Mercedes,7,14,71.428571,6,4
6,1956,Ferrari,10,1956 Ferrari,9,18,55.555556,9,5
7,1957,Maserati,10,1957 Maserati,9,18,55.555556,16,7
8,1958,Ferrari,14,1958 Ferrari,11,22,63.636364,6,3
9,1959,Cooper,13,1959 Cooper,9,18,72.222222,12,5


In [65]:
normalize_podiums["podium_percentage_perdriver_v1"] = (normalize_podiums.podium_percentage) / (normalize_podiums.num_drivers_v1)
normalize_podiums["podium_percentage_perdriver_v2"] = (normalize_podiums.podium_percentage) / (normalize_podiums.num_drivers_v2)

In [66]:
normalize_podiums.head()

,year,constructor_clean,podiums,run,races,available_podium_spots,podium_percentage,num_drivers_v1,num_drivers_v2,podium_percentage_perdriver_v1,podium_percentage_perdriver_v2
0,1950,Alfa Romeo,12,1950 Alfa Romeo,7,14,85.714286,6,4,14.285714,21.428571
1,1951,Alfa Romeo,9,1951 Alfa Romeo,9,18,50.000000,7,4,7.142857,12.500000
2,1952,Ferrari,17,1952 Ferrari,8,16,106.250000,15,6,7.083333,17.708333
3,1953,Ferrari,16,1953 Ferrari,9,18,88.888889,11,4,8.080808,22.222222
4,1954,Mercedes,7,1954 Mercedes,9,18,38.888889,4,3,9.722222,12.962963


In [67]:
normalize_podiums.sort_values(by="podium_percentage_perdriver_v1",ascending=False).head(10)

,year,constructor_clean,podiums,run,races,available_podium_spots,podium_percentage,num_drivers_v1,num_drivers_v2,podium_percentage_perdriver_v1,podium_percentage_perdriver_v2
65,2015,Mercedes,32,2015 Mercedes,19,38,84.210526,2,2,42.105263,42.105263
64,2014,Mercedes,31,2014 Mercedes,19,38,81.578947,2,2,40.789474,40.789474
54,2004,Ferrari,29,2004 Ferrari,18,36,80.555556,2,2,40.277778,40.277778
52,2002,Ferrari,27,2002 Ferrari,17,34,79.411765,2,2,39.705882,39.705882
66,2016,Mercedes,33,2016 Mercedes,21,42,78.571429,2,2,39.285714,39.285714
38,1988,McLaren,25,1988 McLaren,16,32,78.125000,2,2,39.062500,39.062500
61,2011,Red Bull,27,2011 Red Bull,19,38,71.052632,2,2,35.526316,35.526316
51,2001,Ferrari,24,2001 Ferrari,17,34,70.588235,2,2,35.294118,35.294118
43,1993,Williams,22,1993 Williams,16,32,68.750000,2,2,34.375000,34.375000
42,1992,Williams,21,1992 Williams,16,32,65.625000,2,2,32.812500,32.812500


In [68]:
normalize_podiums.sort_values(by="podium_percentage_perdriver_v2",ascending=False).head(10)

,year,constructor_clean,podiums,run,races,available_podium_spots,podium_percentage,num_drivers_v1,num_drivers_v2,podium_percentage_perdriver_v1,podium_percentage_perdriver_v2
16,1966,Brabham,9,1966 Brabham,9,18,50.000000,3,1,16.666667,50.000000
65,2015,Mercedes,32,2015 Mercedes,19,38,84.210526,2,2,42.105263,42.105263
64,2014,Mercedes,31,2014 Mercedes,19,38,81.578947,2,2,40.789474,40.789474
54,2004,Ferrari,29,2004 Ferrari,18,36,80.555556,2,2,40.277778,40.277778
52,2002,Ferrari,27,2002 Ferrari,17,34,79.411765,2,2,39.705882,39.705882
66,2016,Mercedes,33,2016 Mercedes,21,42,78.571429,2,2,39.285714,39.285714
38,1988,McLaren,25,1988 McLaren,16,32,78.125000,2,2,39.062500,39.062500
61,2011,Red Bull,27,2011 Red Bull,19,38,71.052632,2,2,35.526316,35.526316
51,2001,Ferrari,24,2001 Ferrari,17,34,70.588235,2,2,35.294118,35.294118
43,1993,Williams,22,1993 Williams,16,32,68.750000,2,2,34.375000,34.375000


Maybe we should try something different. Currently I'm trying to normalize things in a very flat manner by trying to set an overall number of podium spots. 

What if we go race by race and adjust the number of podium spots based on how many drivers competed in the race. If there were 1 or 2 drivers, the total number of podium spots the team could get in that race were 1 or 2; If there were more than 2, the total number of available spots was limited to three because that's the limit of the podium.

In [69]:
def available_podiums(row):
    season = results[results.year == row.year]
    team = season[season.constructor_clean == row.constructor_clean]
    rounds = team.round_id.unique()
    podium_spots_available = 0
    
    for i in rounds:
        driver_count = team[team.round_id == i].driver.nunique()
        podium_spots_available += min(3,driver_count)
        
    return podium_spots_available

In [70]:
normalize_podiums["available_podium_spots"] = normalize_podiums.apply(available_podiums, axis=1)

In [71]:
normalize_podiums.head(26)

,year,constructor_clean,podiums,run,races,available_podium_spots,podium_percentage,num_drivers_v1,num_drivers_v2,podium_percentage_perdriver_v1,podium_percentage_perdriver_v2
0,1950,Alfa Romeo,12,1950 Alfa Romeo,7,18,85.714286,6,4,14.285714,21.428571
1,1951,Alfa Romeo,9,1951 Alfa Romeo,9,21,50.000000,7,4,7.142857,12.500000
2,1952,Ferrari,17,1952 Ferrari,8,22,106.250000,15,6,7.083333,17.708333
3,1953,Ferrari,16,1953 Ferrari,9,24,88.888889,11,4,8.080808,22.222222
4,1954,Mercedes,7,1954 Mercedes,9,17,38.888889,4,3,9.722222,12.962963
5,1955,Mercedes,10,1955 Mercedes,7,18,71.428571,6,4,11.904762,17.857143
6,1956,Ferrari,10,1956 Ferrari,9,21,55.555556,9,5,6.172840,11.111111
7,1957,Maserati,10,1957 Maserati,9,21,55.555556,16,7,3.472222,7.936508
8,1958,Ferrari,14,1958 Ferrari,11,29,63.636364,6,3,10.606061,21.212121
9,1959,Cooper,13,1959 Cooper,9,24,72.222222,12,5,6.018519,14.444444


In [72]:
normalize_podiums["podium_percentage"] = (normalize_podiums.podiums / normalize_podiums.available_podium_spots)* 100

In [73]:
normalize_podiums.sort_values(by="podium_percentage",ascending=False).head(20)

,year,constructor_clean,podiums,run,races,available_podium_spots,podium_percentage,num_drivers_v1,num_drivers_v2,podium_percentage_perdriver_v1,podium_percentage_perdriver_v2
65,2015,Mercedes,32,2015 Mercedes,19,38,84.210526,2,2,42.105263,42.105263
64,2014,Mercedes,31,2014 Mercedes,19,38,81.578947,2,2,40.789474,40.789474
54,2004,Ferrari,29,2004 Ferrari,18,36,80.555556,2,2,40.277778,40.277778
52,2002,Ferrari,27,2002 Ferrari,17,34,79.411765,2,2,39.705882,39.705882
66,2016,Mercedes,33,2016 Mercedes,21,42,78.571429,2,2,39.285714,39.285714
38,1988,McLaren,25,1988 McLaren,16,32,78.125000,2,2,39.062500,39.062500
2,1952,Ferrari,17,1952 Ferrari,8,22,77.272727,15,6,7.083333,17.708333
61,2011,Red Bull,27,2011 Red Bull,19,38,71.052632,2,2,35.526316,35.526316
51,2001,Ferrari,24,2001 Ferrari,17,34,70.588235,2,2,35.294118,35.294118
43,1993,Williams,22,1993 Williams,16,32,68.750000,2,2,34.375000,34.375000


Let's drop a few of the extra columns and save this.

In [74]:
normalize_podiums = normalize_podiums.drop(columns=["num_drivers_v1", "num_drivers_v2", "podium_percentage_perdriver_v1", "podium_percentage_perdriver_v2"])

In [75]:
normalize_podiums.to_csv("../data/output/podium_count_normalized.csv", index=False)

----

We can actually merge the normalize_podiums and normalize_wins into one data frame and not have to load multiple data frames.

In [76]:
combination = pd.merge(normalize_wins, normalize_podiums, on=["year", "constructor_clean", "run", "races"])

In [77]:
combination.head()

,year,constructor_clean,wins,run,races,win_percentage,podiums,available_podium_spots,podium_percentage
0,1950,Alfa Romeo,6,1950 Alfa Romeo,7,85.714286,12,18,66.666667
1,1951,Alfa Romeo,4,1951 Alfa Romeo,9,44.444444,9,21,42.857143
2,1952,Ferrari,7,1952 Ferrari,8,87.500000,17,22,77.272727
3,1953,Ferrari,7,1953 Ferrari,9,77.777778,16,24,66.666667
4,1954,Mercedes,4,1954 Mercedes,9,44.444444,7,17,41.176471


Neat! let's now go and re-organize these columns:

In [78]:
podiums_and_wins = combination[["year", "constructor_clean", "run", "races", "wins",
                                "podiums", "available_podium_spots", "win_percentage", "podium_percentage"]]

In [79]:
podiums_and_wins.head(10)

,year,constructor_clean,run,races,wins,podiums,available_podium_spots,win_percentage,podium_percentage
0,1950,Alfa Romeo,1950 Alfa Romeo,7,6,12,18,85.714286,66.666667
1,1951,Alfa Romeo,1951 Alfa Romeo,9,4,9,21,44.444444,42.857143
2,1952,Ferrari,1952 Ferrari,8,7,17,22,87.500000,77.272727
3,1953,Ferrari,1953 Ferrari,9,7,16,24,77.777778,66.666667
4,1954,Mercedes,1954 Mercedes,9,4,7,17,44.444444,41.176471
5,1955,Mercedes,1955 Mercedes,7,5,10,18,71.428571,55.555556
6,1956,Ferrari,1956 Ferrari,9,5,10,21,55.555556,47.619048
7,1957,Maserati,1957 Maserati,9,4,10,21,44.444444,47.619048
8,1958,Ferrari,1958 Ferrari,11,2,14,29,18.181818,48.275862
9,1959,Cooper,1959 Cooper,9,5,13,24,55.555556,54.166667


In [80]:
podiums_and_wins.to_csv("../data/output/wins_and_podiums.csv", index=False)